In [1]:
from __future__ import annotations

from pathlib import Path

import pandas as pd

In [2]:
csv_file_dir = Path("../../content/tables/TRP/")
params_to_keep = set(
    ["CRRA", "BeqFac", "BeqShift", "WealthShare", "WealthShift", "criterion"],
)

# Get all files in the directory
files = [f for f in csv_file_dir.iterdir() if f.is_file()]

parameters = []

# Iterate over each file
for file in files:
    file_name = file.stem.replace("_estimate_results", "")
    # Read the CSV file and convert it to a dictionary
    res = pd.read_csv(file, header=None)
    res = res.set_index(res.columns[0])[res.columns[1]].to_dict()

    # Create a new dictionary for this file
    file_parameters = {}

    labor = 0
    stock = 0

    file_parameters["Name"] = file_name

    # Iterate over each parameter we want to keep
    for param in params_to_keep:
        # If the parameter is in the dictionary, add it to the file's parameters
        if param in res:
            file_parameters[param] = res[param]

    # Add the file's parameters to the overall parameters dictionary
    parameters.append(file_parameters)

In [3]:
def format_df(df):
    for col in df.columns:
        if col == "Name":
            continue
        # Check if column is of float type
        if col in params_to_keep:
            df[col] = df[col].astype(float).round(3).fillna("")
        # Check if column contains only 0 and 1
        else:
            df[col] = df[col].map({0: "", 1: "✔️"})
    return df


# Define the order of columns
column_order = ["Name", "criterion", "CRRA", "WealthShare", "BeqFac", "BeqShift"]

df = pd.DataFrame(parameters)
formatted_df = format_df(df)[column_order].sort_index()
formatted_df


# Life cycle portfolio choice
# Bequest portfolio Choice
# TRP Life cycle portfolio choice
# leave out wealth shift

,Name,criterion,CRRA,WealthShare,BeqFac,BeqShift
0,Portfolio,0.642,9.252,,,
1,WarmGlowPortfolio,0.641,9.207,,23.051,45.643
2,WealthPortfolio,0.242,5.336,0.171,,


In [4]:
formatted_df.to_latex("../../content/tables/parameters.tex", index=False)